# Diagonalização dos operadores para cálculo da Ação Efetiva para QED + Torção

Nas contas, encontramos para a ação efetiva $\bar{\Gamma}[A,S]$ a expressão
\begin{equation}
 \bar{\Gamma} [A,S] = - \frac{i}{2} {\rm Tr} \ln{\left\{ \frac{1}{P^2 - m^2} \left[\left(P_{\mu} + e A_{\mu} - \eta S_{\mu} \gamma_5 \right)^2 + \frac{1}{2} \left(e F^{\mu \nu} - \eta S^{\mu \nu} \gamma_5\right) \sigma_{\mu \nu} - 2 \eta m S^{\mu} \gamma_{\mu} \gamma_5 - m^2\right]\right\}}.
\end{equation}
Para calcular a ação efetiva, é preciso primeiro diagonalizar o operador no traço. Abrindo a expressão no primeiro termo, é possível encontrar um termo proporcional à idêntidade em conjunto com o termo $\mathbf{\nabla} \cdot \mathbf{S} \gamma_5$. Utilizando então as identidades:
\begin{align}
\gamma^{\mu} \gamma_5 = \frac{1}{6} \varepsilon^{\mu \nu \alpha \beta} \gamma_{\nu} \sigma_{\alpha \beta} & & \text{ e } & & \gamma_5 \sigma^{\mu \nu} = \frac{i}{2} \varepsilon^{\mu \nu \alpha \beta} \sigma_{\alpha \beta},
\end{align}
Obtem-se então o termo
\begin{align}
 \bar{\Gamma} [A,S] & = - \frac{i}{2} {\rm Tr} \ln{\left\{ \left(P_{\mu} - e A_{\mu} \right)^2 + \eta^2 S^2 - m^2 - \eta \left[i \left(\partial_\mu S^\mu \right) - 2 e A_\mu S^\mu - 2 i S^\mu \partial_\mu \right] \gamma_5 + \frac{1}{2} \left(V^{\mu \nu} - \frac{2}{3} \varepsilon^{\alpha \beta \mu \nu} S_{\alpha} \gamma_{\beta} \right) \sigma_{\mu \nu}\right\}} + \frac{i}{2} {\rm Tr} \ln{\left(P^2 - m^2 \right)}, \\
 & = - \frac{i}{2} {\rm Tr} \ln{\left\{ \left[ \left(P_{\mu} - e A_{\mu}\right)^2 + \eta^2 S^2_{\mu} - m^2\right] \mathbf{1} - \mathcal{A} \gamma_5 + \frac{1}{2} \left(V^{\mu \nu} - \frac{2}{3} \varepsilon^{\alpha \beta \mu \nu} S_{\alpha} \gamma_{\beta} \right) \sigma_{\mu \nu} \right\}} + \frac{i}{2} {\rm Tr} \ln{\left( P^2 - m^2 \right)},
\end{align}
onde $\mathbf{I}$ é a matriz identidade, mudamos a escala do vetor axial por conveniência matemática, ou seja, $\mathbf{S} \to \eta m \mathbf{S}$. Como o primeiro termo é proporcional à matriz identidade, basta diagonalizar os demais termos, para que possamos calcular a ação efetiva. Faremos isso passo a passo para garantir que o programa funciona adequadamente, começando por importar os pacotes necessários e definindo as variáveis simbólicas.

In [103]:
# Autor: Lucas Pereira de Souza

import sympy as sp
import sympy.physics.vector as spv
import sympy.physics.matrices as spm
from sympy.physics.hep.gamma_matrices import GammaMatrix as G, LorentzIndex
from sympy.tensor.tensor import TensorIndexType, TensorIndex, TensorHead, tensor_indices, TensorSymmetry
from collections import Counter
from IPython.display import display, Math
import warnings
warnings.filterwarnings("ignore")

# variáveis simbólicas. a = div(S), s_u são componentes de s, v_ab são componentes do tensor antisimétrico.
Evx, Evy, Evz, Bvx, Bvy, Bvz, E, B = sp.symbols('E^{(V)}_{x} E^{(V)}_{y} E^{(V)}_{z} B^{(V)}_{x} B^{(V)}_{y} B^{(V)}_{z} E_V B_V')
s0, s1, s2, s3, S, n, m = sp.symbols('S_t S_x S_y S_z S \eta m')
Ev, Bv, Sj, V, W, tVcS = sp.symbols(r'\vec{E}_V \vec{B}_V \vec{S} \mathcal{V} \mathcal{W} \tilde{V}^{\mu\nu}S_\nu')
a = sp.symbols('(\Pi_{\mu}S^{\mu})')
SxE, SxB, SdotB, SdotE, BdotSxE = sp.symbols(r'\left(\vec{S}\times\vec{E}_V\right) \left(\vec{S}\times\vec{B}_V\right) \left(\vec{S}\cdot\vec{B}_V\right) \left(\vec{S}\cdot\vec{E}_V\right) \vec{B}_V\cdot\left(\vec{S}\times\vec{E}_V\right)')
lamb = sp.symbols('\lambda') # Variável

print("Torsion Diagonalization.ipynb"+"\n")
print("=" * 100)
print("Autor: Lucas Pereira de Souza")
print("=" * 100)
print("\n"+"Este programa diagonaliza algumas das combinações de elementos que surgem")
print("nas Hessianas do férmion acoplado ao campo axial. Para rodar, basta apertar")
print("os botões no canto superior esquerdo em cada seção do notebook.")
print("\n"+"Vamos começar definindo as funções e Matrizes com as quais trabalharemos:"+"\n")

Torsion Diagonalization.ipynb

Autor: Lucas Pereira de Souza

Este programa diagonaliza algumas das combinações de elementos que surgem
nas Hessianas do férmion acoplado ao campo axial. Para rodar, basta apertar
os botões no canto superior esquerdo em cada seção do notebook.

Vamos começar definindo as funções e Matrizes com as quais trabalharemos:



In [104]:
# O python já tem código pra isso, mas ele não estava conseguindo resolver alguns casos, então tive que escrever a função à mão
def resolver_equacao_caracteristica(M, lam, invarlist):
  charpoly = M.charpoly(lamb)
  expr0 = charpoly.as_expr()
  expr = sp.collect(sp.simplify(sp.collect_const(sp.simplify((sp.expand(expr0))))),lamb)#.subs(invariantes)

  # Resolve a equação característica em C
  if invarlist==None:
    expr_simp = sp.collect(sp.simplify(sp.collect_const(sp.expand(expr))),lamb)
  else:
    for i in range (len(invariantes)):
      expr0_simp = sp.expand(expr0).subs(invariantes)
    expr_simp = sp.collect(sp.simplify(sp.collect_const(sp.expand(expr0_simp).subs(invariantes))),lamb)

  #print("\n" + "=" * 100)
  #print(r"Polinômio Característico antes da simplificação:")
  #print("\n" + "=" * 100)
  #display(Math(sp.latex(expr)))

  print("\n" + "=" * 100)
  print(r"Polinômio Característico após simplificação:")
  print("\n" + "=" * 100)
  display(Math(sp.latex(expr_simp)))

  raizes = sp.solve(expr_simp, lamb, domain=sp.S.Complexes)
  n = 0
  while n<5:
    if invarlist==None:
      raizes_quase = [sp.expand(r) for r in raizes]
      raizes_simpl=[sp.simplify(sp.collect_const(sp.simplify((sp.expand(r))))) for r in raizes_quase]
    else:
      raizes_quase = [(sp.expand(r).subs(invariantes)) for r in raizes]
      raizes_simpl = [sp.simplify(sp.collect_const(sp.simplify((sp.expand(r).subs(invariantes))))) for r in raizes_quase]
    n = n+1
    contagem = Counter(raizes_simpl)

  #raizes_debug = sp.solve(sp.expand(expr), lamb, domain=sp.S.Complexes)
  #raizes_debug = [sp.simplify(sp.collect_const(sp.simplify((sp.expand(r))))) for r in raizes_debug]
  #contagem_debug = Counter(raizes_debug)

  # Exibe autovalores e multiplicidades algébricas
  #print("\n" + "=" * 100)
  #print(r"Autovalores antes a simplificação: ") #e multiplicidades Algébricas antes da simplificação: ")
  #print("\n" + "=" * 100)
  #for r, m in contagem_debug.items():
    #display(Math(r'\lambda = %s' % (sp.latex(r))))
    #display(Math(r'm_{alg} = %d' % (m)))
  #print("\n" + "=" * 100)

  # Exibe autovalores e multiplicidades algébricas
  print("\n" + "=" * 100)
  print(r"Autovalores após simplificação: ")
  print("\n" + "=" * 100)
  for r, m in contagem.items():
    display(Math(r'\lambda = %s' % (sp.latex(r))))
    #display(Math(r'm_{alg} = %d' % (m)))
  print("\n" + "=" * 100)

  print("\n" + "=" * 100)



In [112]:
# Definição das matrizes simbólicas com as quais vamos trabalhar:
# V^ab sigma_ab
#Vsigma = 2*sp.Matrix([
#[v12, v23 - sp.I*v31, v03*sp.I, v02 + v01*sp.I],
#[v23 + sp.I*v31, -v12, -v02+v01*sp.I, -v03*sp.I],
#[v03*sp.I, v02 + v01*sp.I, v12, v23-sp.I*v31],
#[-v02+v01*sp.I, -v03*sp.I, v23+sp.I*v31,-v12]
#])

def sigma(a,b):
  return (sp.I/2)*(spm.mgamma(a)*spm.mgamma(b) - spm.mgamma(b)*spm.mgamma(a))

Vmatrix = sp.Matrix([
  [0,Evx,Evy,Evz],
  [-Evx,0,-Bvz,Bvy],
  [-Evy,Bvz,0,-Bvx],
  [-Evz,-Bvy,Bvx, 0]
])

tildeVmatrix = sp.Matrix([
  [0,-Bvx,-Bvy,-Bvz],
  [Bvx,0,Evz,-Evy],
  [Bvy,-Evz,0,Evx],
  [Bvz,Evy,-Evx, 0]
])
eta = sp.Matrix.diag(1,-1,-1,-1)

print("=" * 100)
# Show the matrix in LaTeX
display(Math(r"V^{\mu \nu} = "+f"{sp.latex(Vmatrix)}"))
print("\n" + "=" * 100)

Vsigma = sp.zeros(4,4)
nmSgammag5 = sp.zeros(4,4)
vecs = sp.Matrix([[s0],[s1],[s2],[s3]])
tvecs = tildeVmatrix*vecs
nmtvecs2 = 0
for i in range(4):
  nmSgammag5 = nmSgammag5 + n*m*vecs[i]*(spm.mgamma(i)*spm.mgamma(5))
  if i == 0:
    nmtvecs2 = nmtvecs2 + ((n*m)**2)*tvecs[i]*tvecs[i]
  else:
    nmtvecs2 = nmtvecs2 - ((n*m)**2)*tvecs[i]*tvecs[i]
  for j in range(4):
    Vsigma = Vsigma + Vmatrix[i,j]*sigma(i,j)

#tvecs2 = Vsigma*epsilonSgamma + epsilonSgamma*Vsigma
#tvai = tvecs2*tvecs2

# Step 1: Display the matrix
print("=" * 100)
# Show the matrix in LaTeX
display(Math(r"V^{\mu \nu} \sigma_{\mu \nu} = "+f"{sp.latex(Vsigma)}"))
print("\n" + "=" * 100)

#epsilonSgamma = -sp.Matrix([
#[s3, s1 - sp.I*s2, -s0, 0],
#[s1 + sp.I*s2, -s3, 0, -s0],
#[s0, 0, -s3, -s1+sp.I*s2],
#[0, s0, -s1-sp.I*s2,s3]
#])

print("=" * 100)
display(Math(r" \eta m S_{\mu} \gamma^{\mu} \gamma^5 = "+f"{sp.latex(nmSgammag5)}"))
print("\n" + "=" * 100)

H5 = n*a*spm.mgamma(5)#sp.Matrix([
#[0, 0, a, 0],
#[0, 0, 0, a],
#[a, 0, 0, 0],
#[0, a, 0,0]
#])

print("=" * 100)
display(Math(r"\eta \Pi_\mu S^\mu \gamma_5 = "+f"{sp.latex(H5)}"))
print("\n" + "=" * 100)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [163]:
# Termos para simplificar a exibição, ao invés de exibir cada componente de cada vetor e tensor
N = spv.ReferenceFrame('N')
Evec = Evx*N.x + Evy*N.y + Evz*N.z
Bvec = Bvx*N.x + Bvy*N.y + Bvz*N.z
Svec = s1*N.x + s2*N.y + s3*N.z
Ev2 = Evec.dot(Evec)
Bv2 = Bvec.dot(Bvec)
Sv2 = Svec.dot(Svec)
SdotEv = Svec.dot(Evec)
SdotBv = Svec.dot(Bvec)
SxEv = -Svec.cross(Evec) #s2*v03 - s3*v02 + s3*v01 - s1*v03 + s1*v02 - s2*v01
SxBv = Svec.cross(Bvec) #s2*v12 - s3*v31 + s3*v23 - s1*v12 + s1*v31 - s2*v23
SoBdotSxE = s0*Bvec.dot(Svec.cross(Evec)) # s0*s1*v02*v12 - s0*s1*v03*v31 - s0*s2*v01*v12 + s0*s2*v03*v23 + s0*s3*v02*v23 - s0*s3*v02*v23
Coolexpr = (SxEv - s0*Bvec).dot(SxEv - s0*Bvec)

Sinv = s0**2 - Sv2
nmSinv = ((n*m)**2)*Sinv #s1**2 - s2**2 - s3**2
Vinv = sp.Rational(1,2)*sp.expand((Bv2 - Ev2))
Winv = Bvec.dot(Evec)

Sinv2 =sp.simplify(sp.expand(Sinv**2))
Vinv2 = sp.collect_const(sp.expand(Vinv**2))
nmSinv2 = sp.simplify(sp.expand(nmSinv**2))
Winv2 = sp.collect_const(sp.expand(Winv**2))
VWinv = sp.collect_const(sp.expand(Vinv*Winv))

Fp = sp.sqrt(2*Vinv + 2*sp.I*Winv)
Fm = sp.sqrt(2*Vinv - 2*sp.I*Winv)

FpFm = sp.expand(Fp*Fm)
FpFm2 = sp.expand((Fp*Fm)**2)
V2l2 = sp.expand(Vinv*(lamb**2))
WAl = sp.expand(16*sp.I*(n*a)*Winv*lamb)
VA2 = sp.expand(16*Vinv*(n*a**2))
AV12 = sp.expand(8*(lamb**2)*((n*a)**2) + 4*(lamb**2)*Vinv)
resto = sp.expand(16*(n*a)**4 - 16*((n*a)**2)*Vinv + 4*Winv**2 + 4*Vinv**2)

ssV = sp.expand(nmSinv*Vinv)
ssW = sp.expand(nmSinv*Winv)
Xp = sp.expand(sp.sqrt(2*Vinv - 4*nmSinv + 2*sp.sqrt(4*nmtvecs2 - Winv**2)))
Xm = sp.expand(sp.sqrt(2*Vinv - 4*nmSinv - 2*sp.sqrt(4*nmtvecs2 - Winv**2)))
XpXm = sp.expand(Xp*Xm)
XpXm2 = sp.expand((XpXm)**2)
ssmA2 = sp.expand(16*((n*a)**2 - nmSinv)**2)
sl2 = sp.expand(8*nmSinv*(lamb**2))
sa2 = sp.expand(16*nmSinv*((n*a)**2))
resto2 = sp.expand(- 16*((n*a)**2)*(Vinv + 2*nmSinv))

# Dicionário de invariantes para substituir nas expressões, sempre que identificar os termos semelhantes
invariantes = {Vinv: V, Winv: W, -Vinv: -V, sp.expand(-4*Vinv*lamb**2): -4*V*(lamb**2), sp.sqrt(sp.expand(-Winv2)): sp.I*W, sp.I*sp.expand(Winv): sp.I*W, sp.expand(Fp): sp.sqrt(2*V + 2*sp.I*W), sp.expand(Fm): sp.sqrt(2*V - 2*sp.I*W) ,
               sp.expand(FpFm2): 4*(V**2 + W**2), sp.expand(Winv2): W**2, sp.expand(Vinv2): V**2,
               V2l2:V*(lamb**2), sp.expand(WAl/16): sp.I*n*a*W*lamb, sp.expand(WAl): 16*sp.I*n*a*W*lamb, AV12: (8*(n*a)**2 + 4*V)*lamb**2, VA2: 4*V*((n*a)**2), resto: 16*((n*a)**2)*((n*a)**2 - V) + 4*(V**2 + W**2),
               nmtvecs2: (n*m*tVcS)**2, sp.simplify(16*nmtvecs2): 16*(n*m*tVcS**2),
               Xp: sp.sqrt(2*V - 4*(n*m*S)**2 + 2*sp.sqrt(4*(n*m*tVcS)**2 - W**2)),
               Xm: sp.sqrt(2*V - 4*(n*m*S)**2 - 2*sp.sqrt(4*(n*m*tVcS)**2 - W**2)),
               XpXm2: (16*(n*m*S)**4 - 16*V*(n*m*S)**2 + 4*V**2 + 4*W**2 - 16*(n*m*tVcS)**2), sp.expand(16*nmSinv2): 16*(n*m*S)**4, sp.expand(4*nmSinv): 4*(n*m*S)**2, sp.expand(Sinv):S**2, sp.expand(16*ssV):16*V*(n*m*S)**2,
               sp.expand(-16*(nmtvecs2 + ssV)):-16*(n**2)*((m*tVcS)**2 + V*((m*S)**2)), sp.expand(-16*(nmtvecs2 - ssV)):-16*(n**2)*((m*tVcS)**2 - V*((n*m*S)**2)),
               sl2: 8*((n*m*S)**2)*(lamb**2), sa2: 16*(n**4)*((m*S)**2)*(a**2), sp.expand(ssmA2): 16*((n*a)**2 - (n*m*S)**2)**2, resto2:-16*((n*a)**2)*(V + 2*((n*m)**2)*S)}

## 1 - Caso Eletromagnético

Quando $V^{\mu \nu} = F^{\mu \nu}$. Neste caso, observe que queremos diagonalizar
\begin{equation}
\begin{gathered}
\frac{1}{2} V^{\mu \nu} \sigma_{\mu \nu} = \frac{1}{2} \begin{pmatrix}
\epsilon^{ijk} V_{ij} \sigma_k & i V^{0j} \sigma_j \\
i V^{0 j} \sigma_j & \epsilon^{ijk} V_{ij} \sigma_k
\end{pmatrix}, \\
 = \begin{pmatrix}
V^{12} & V^{23} - i V^{13} & i V^{03} & V^{02} + i V^{01}\\
V^{23} + i V^{13} & - V^{12} & -V^{02} + i V^{01} & -i V^{03} \\
i V^{03} & V^{02} + i V^{01} & V^{12} & V^{23} - i V^{13} \\
-V^{02} + i V^{01} & -i V^{03} & V^{23} + i V^{13} & - V^{12}
\end{pmatrix},
\end{gathered}
\end{equation}
sendo $\sigma_j$ as matrizes de Pauli. Já fiz o processo algébrico, calculando os invariantes à partir de $(V^{\mu \nu} \sigma_{\mu \nu})^2$ e obtivemos
\begin{equation}
\left(V^{\mu \nu} \sigma_{\mu \nu}\right)_{D} = \begin{pmatrix}
\sqrt{2 \left(\mathcal{V} + i \mathcal{W} \right)} \sigma_3 & 0 \\
0 & \sqrt{2 \left(\mathcal{V} - i \mathcal{W} \right)} \sigma_3
\end{pmatrix},
\end{equation}
onde
\begin{align}
\mathcal{V} & = \frac{1}{4} V^2_{\mu \nu} = \frac{1}{2} \left(V^2_{12} + V^2_{23} + V^2_{31} - V^2_{01} - V^2_{02} - V^2_{03} \right) = \frac{1}{2} \left(\mathbf{B}^2_V - \mathbf{E}^2_V\right), \\
\mathcal{W} & = - \frac{1}{4} \tilde{V}^{\mu \nu} V_{\mu \nu} = \sqrt{\det{(V_{\mu \nu})}} = \left(V_{01} V_{23} + V_{02} V_{31} + V_{03} V_{12} \right) = \left(\mathbf{E}_V \cdot \mathbf{B}_V\right),
\end{align}
onde definimos os campos $\mathbf{E}_V$ e $\mathbf{B}_V$ em analogia com os campos eletromagnéticos usuais. Agora vamos conferir se o programa acerta:

In [140]:
auts0 = resolver_equacao_caracteristica(sp.Rational(1,2)*Vsigma, lamb, invariantes)


Polinômio Característico após simplificação:



<IPython.core.display.Math object>


Autovalores após simplificação: 



<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## 2 - Torção Transversa

Neste caso, tomamos $\mathbf{\nabla} \cdot \mathbf{S} = 0$, isto é, precisamos diagonalizar o termo $\left(V^{\mu \nu} + \frac{2}{3} \varepsilon^{\alpha \beta \mu \nu} S_{\alpha} \gamma_{\beta} \right) \sigma_{\mu \nu}$. Para isso, note que
\begin{gather}
\varepsilon^{\alpha \beta \mu \nu} S_{\alpha} \gamma_{\beta} \sigma_{\mu \nu} = 6 \begin{pmatrix}
-S^j \sigma_j & S^0 \mathbf{I} \\
-S_0 \mathbf{I} & S^j \sigma_j
\end{pmatrix}, \\
= 6 \begin{pmatrix}
-S^3 & -S^1 + i S^2 & S^0 & 0 \\
-S^1 - i S^2 & S^3 & 0 & S^0 \\
-S^0 & 0 & S^3 & S^1 - i S^2 \\
0 & - S^0 & S^1 + i S^2 & -S^3
\end{pmatrix},
\end{gather}
de forma que
\begin{gather}
\left(V^{\mu \nu} + \frac{2}{3} \varepsilon^{\alpha \beta \mu \nu} S_{\alpha} \gamma_{\beta} \right) \sigma_{\mu \nu} = \begin{pmatrix}
\left(\epsilon^{ijk} V_{ij} - 4 S^k \right) \sigma_k & + 4 S^0 \mathbf{I} + i V^{0j} \sigma_j \\
-4 S^0 \mathbf{I} + i V^{0 j} \sigma_j & \left(\epsilon^{ijk} V_{ij} + 4 S^k \right) \sigma_k
\end{pmatrix}, \\
= 2 \begin{pmatrix}
V^{12} - 2 S^3 & V^{23} - 2 S^1 - i (V^{13} - S^2) & S^0 + i V^{03} & V^{02} + i V^{01}\\
V^{23} - 2 S^1 + i (V^{13} - S^2) & - V^{12} + 2 S^3 & -V^{02} + i V^{01} & S^0 - i V^{03} \\
-S^0 + i V^{03} & V^{02} + i V^{01} & V^{12} + 2 S^3 & V^{23} + 2 S^1 - i (V^{13} + S^2) \\
-V^{02} + i V^{01} & - S^0 - i V^{03} & V^{23} - 2 S^1 + i (V^{13} - S^2) & - V^{12} - 2 S^3
\end{pmatrix},
\end{gather}
e vamos ver se o resultado que o programa obtém é o correto encontrado algebricamente.

Vamos definir ainda
\begin{equation}
S^2 = S^2_0 - S^2_1 - S^2_2 - S^2_3,
\end{equation}
bem como os eventuais produtos
\begin{align}
\mathbf{S} \cdot \mathbf{E}_V = S^1 V^01 + S^2 V^02 + S^3 V^03 & & \text{ e } & & \mathbf{S} \cdot \mathbf{B}_V = S^1 V^{23} + S^2 V^{31} + S^3 V^{12} = \frac{1}{2} \epsilon_{ijk} S^i V^{jk},
\end{align}
assim como
\begin{equation}
\left|\mathbf{S} \times \mathbf{E}_V\right|^2 = \epsilon_{ijk} S^i V^{0j} = S^2 V^{03} - S^3 V^{02} + S^3 V^{01} - S^1 V^{03} + S^1 V^{02} - S^2 V^{01},
\end{equation}
e
\begin{align}
\left|\mathbf{S} \times \mathbf{B}_V \right|^2 & = \frac{1}{2} \varepsilon^{i j k} \varepsilon^{j n m} S^i V^{n m}, \\
& = - \left(\delta^{in} \delta^{km} - \delta^{im} \delta^{kn} \right) S^i V^{n m} = - \sum_{i \neq j} S^i V^{i j}, \\
& = S^2 V^{12} - S^3 V^{31} + S^3 V^{23} - S^1 V^{12} + S^1 V^{31} - S^2 V^{23},
\end{align}
além de
\begin{equation}
\left(\tilde{V}^{\mu \nu} S_\nu \right)^2 = \left(\vec{S} \times \vec{E}_V \right)^2 + \left(S_0 \vec{B}_V\right)^2 - 2 S_0 \vec{B}_V \cdot \left(\vec{S} \times \vec{E}_V\right) - \left(\vec{S} \cdot \vec{B}_v \right)^2 = - 2 S^2 \mathcal{V} + \left(S_\nu V^{\mu \nu} \right)^2,
\end{equation}
para que os termos sejam simplificados.

In [160]:
H1 = sp.Rational(1,2)*Vsigma + 2*nmSgammag5
print("=" * 50)
# Show the matrix in LaTeX
display(Math(r"\frac{1}{2} V_{\rho \lambda} \sigma^{\rho \lambda} + 2 \eta m S_{\mu} \gamma^{\nu} \gamma^5 = "+f"{sp.latex(H1)}"))
print("\n" + "=" * 50)
#display(Math(r"16 \eta^2 m^2 \left(\tilde{V}^{\mu \nu} S_\nu \right)^2 = "+f"{sp.latex(sp.expand(16*nmtvecs2))}"))
auts1 = resolver_equacao_caracteristica(H1,lamb,invariantes)

<IPython.core.display.Math object>



Polinômio Característico após simplificação:



<IPython.core.display.Math object>


Autovalores após simplificação: 



<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## 3 - Torção Vinculada ao Campo Eletromagnético

Este consiste de assumir que $2 \Pi_\mu S^\mu = \left(\partial_\mu S^\mu - 2 S^\mu \partial_\mu - 2 i e A_\mu S^\mu \right) \neq 0$, mas $V_{\mu \nu} = 0$, isto é, $F_{\mu \nu} = i \eta \tilde{S}_{\mu \nu}$. Neste caso teremos então

In [143]:
H2 = 2*H5 + 2*nmSgammag5
print("=" * 50)
# Show the matrix in LaTeX
display(Math(r"2 \eta \left(\Pi_\mu S^\mu \mathbf{1} + m S_{\mu} \gamma^{\mu} \right) \gamma^5 = " + f"{sp.latex(H2)}"))
print("\n" + "=" * 50)
auts2 = resolver_equacao_caracteristica(H2,lamb,invariantes)

<IPython.core.display.Math object>



Polinômio Característico após simplificação:



<IPython.core.display.Math object>


Autovalores após simplificação: 



<IPython.core.display.Math object>

<IPython.core.display.Math object>

## 4 - Relatividade Muito Especial

Neste caso, tomamos $S_\mu \gamma^\mu \gamma^5 \psi = 0$, que só é possível se $\det{\left(S_\mu \gamma^\mu \gamma^5 \right)} = 0$, isto é, $-S^2 = 0$.

In [164]:
H3 = 2*H5 + sp.Rational(1,2)*Vsigma
print("=" * 50)
# Show the matrix in LaTeX
display(Math(r"2 \eta \Pi_\mu S^\mu \gamma_5 + \frac{1}{2} V_{\mu \nu} \sigma^{\mu \nu} = " + f"{sp.latex(H3)}"))
print("\n" + "=" * 50)
auts3 = resolver_equacao_caracteristica(H3,lamb,invariantes)

<IPython.core.display.Math object>



Polinômio Característico após simplificação:



<IPython.core.display.Math object>


Autovalores após simplificação: 



<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## 5 - Caso Geral

Este consiste de assumir que nenhum termo zera.
\begin{gather}
2 \eta \Pi_\mu S^\mu \gamma_5 + \frac{1}{2} V^{\mu \nu} \sigma_{\mu \nu} + 2 \eta m S_{\mu} \gamma^{\mu} \gamma^5 = \begin{bmatrix}
\left(\epsilon^{ijk} V_{ij} - 4 \eta m S^k \right) \sigma_k & 2 \eta \left(2 m S^0 + \Pi_\mu S^\mu \right) \mathbf{I} + i V^{0j} \sigma_j \\
2 \eta \left(\Pi_\mu S^\mu - 2 m S^0 \right) \mathbf{I} + i V^{0 j} \sigma_j & \left(\epsilon^{ijk} V_{ij} + 4 \eta m S^k \right) \sigma_k
\end{bmatrix},
\end{gather}

In [165]:
H4 = 2*H5 + sp.Rational(1/2)*Vsigma + 2*nmSgammag5
print("=" * 50)
# Show the matrix in LaTeX
display(Math(r"2 \eta \left(\Pi_\mu S^\mu \right) \gamma_5 + \frac{1}{2} V_{\mu \nu} \sigma^{\mu \nu} + 2 \eta m S_{\mu} \gamma^{\mu} \gamma^5 = " + f"{sp.latex(H4)}"))
print("\n" + "=" * 50)
auts4 = resolver_equacao_caracteristica(H4,lamb,invariantes)

#geos = multiplicidade_geometrica(M2, auts)

#cert = certificado_nao_diagonalizacao(M2, auts)

<IPython.core.display.Math object>



Polinômio Característico após simplificação:



<IPython.core.display.Math object>


Autovalores após simplificação: 



<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## RASCUNHO

In [ ]:
# RASCUNHO
# variáveis simbólicas. a = div(S), s_u são componentes de s, v_ab são componentes do tensor antisimétrico.
a, vinv, winv, Ev, Bv = sp.symbols('a Vinv Winv Ev Bv')
Evx, Evy, Evz, Bvx, Bvy, Bvz, St, Sx, Sy, Sz = sp.symbols('Ev_x Ev_y Ev_z Bv_x Bv_y Bv_z St Sx Sy Sz')
N = spv.ReferenceFrame('N')
Evexpr = Evx*N.x + Evy*N.y + Evz*N.z
Bvexpr = Bvx*N.x + Bvy*N.y + Bvz*N.z
Svec = Sx*N.x + Sy*N.y + Sz*N.z
k = St**2 - Sx**2 - Sy**2 - Sz**2
Lorentz = TensorIndexType('Lorentz', dummy_name='L')
#Lorentz.metric = Lorentz.get_metric(-1)
repl = {Lorentz: sp.diag(1, -1, -1, -1)}
j = tensor_indices('j', LorentzIndex)
i0, i1, i2, i3 = tensor_indices('i0:4', Lorentz)
#mu = TensorIndex('mu', Lorentz, is_up=False)
#nu, rho = tensor_indices('nu, rho', Lorentz)
S = TensorHead('S', [Lorentz])
repl.update({S(i0): [St, Sx, Sy, Sz]})
asym2 = TensorSymmetry.fully_symmetric(-2)
V = TensorHead('V', [Lorentz, Lorentz], asym2)
V_i0_i1 = V(-i0,-i1)
S_i0 = S(i0)
tVS = TensorHead(r'\tilde{V}^{\mu\nu}S_\nu', [Lorentz])
repl.update({V(-i0, -i1): [
[0, Evx, Evy, Evz],
[-Evx, 0, -Bvz, Bvy],
[-Evy, Bvz, 0, -Bvx],
[-Evz, -Bvy, Bvx, 0]
]})
W = TensorHead('W', [Lorentz, Lorentz], asym2)
repl.update({W(-i0, -i1): [
[0, Bvx, Bvy, Bvz],
[-Bvx, 0, Evz, -Evy],
[-Bvy, -Evz, 0, Evx],
[-Bvz, Evy, -Evx, 0]
]})
expr = sp.collect_const((W(i0,i1)*V(-i0,-i1)).replace_with_arrays(repl, []))
expr2 = sp.collect_const((W(-i0,-i1)*S(i1)).replace_with_arrays(repl, []))
tVS2 = sp.collect_const(((expr2(-i0)*expr2(i0))).replace_with_arrays(repl, []))
EBexp = Evexpr.dot(Bvexpr)

display(Math(r"V^{\mu \nu} = "+f"{sp.latex((V(i0, i1)).replace_with_arrays(repl, [i0, i1]))}"))
display(Math(sp.latex(sp.simplify((V(i0,i1)*V(-i0,-i1)).replace_with_arrays(repl, [])))))
display(Math(sp.latex(expr.subs(EBexp, "EB_V"))))
display(Math(f"S = {sp.latex((S(-i0)).replace_with_arrays(repl, [i0]))}"))
display(Math(f"S^2 = {sp.latex((S(i0)*S(-i0)).replace_with_arrays(repl, []))}"))
display(Math(sp.latex(sp.simplify((S(i0)*S(-i0)).replace_with_arrays(repl, [])))))
display(Math(sp.latex(tVS(-i0)).replace_with_arrays(repl, [])))
display(Math(r"\left(\tilde{V}_{\mu \nu} S^\nu\right)^2 = "+f"{sp.latex(tVS2)}"))
display(Math(f"Ev = {sp.latex(Ev)}"))
display(Math(f"Bv = {sp.latex(Bv)}"))
display(Math(f"Svec = {sp.latex(Svec)}"))
display(Math(f"Ev.Bv = {sp.latex(Evexpr.dot(Bvexpr))}"))
display(Math(r"Ev \times Bv ="+f"{sp.latex(Evexpr.cross(Bvexpr))}"))

#---------------------------------------------------------
# Função 2: Multiplicidade geométrica de cada autovalor
#---------------------------------------------------------
def multiplicidade_geometrica(M, autovalores):
    n = M.shape[0]
    display(Math(r"\textbf{Multiplicidades Geométricas:}"))

    resultados_geo = {}
    for val, m_alg in autovalores.items():
        try:
            N = (M - val * sp.eye(n)).applyfunc(sp.simplify)
            nullspace = N.nullspace()
            m_geo = len(nullspace)
        except Exception as e:
            display(Math(r"\lambda = "+f"{sp.latex(val)}"+r",\quad \textcolor{{red}}{{\text{{Erro ao calcular: {str(e)} }}}}"))
            m_geo = 0

        resultados_geo[val] = m_geo

        display(Math(r"\lambda = %s,\quad m_{geo} = %d" % (sp.latex(val), m_geo)))
        if m_geo < m_alg:
            display(Math(r"\textcolor{red}{\text{Não diagonalizável neste autovalor}}"))
        else:
            display(Math(r"\textcolor{green}{\text{Diagonalizável neste autovalor}}"))

    return resultados_geo


#---------------------------------------------------------
# Função 3: Certificado simbólico de não-diagonalização
# Constrói forma de Jordan e exibe bloco(s) não triviais
#---------------------------------------------------------
def certificado_nao_diagonalizacao(M, autovalores=None):
    # Verifica se a matriz é diagonalizável
    if M.is_diagonalizable:
        display(Math(r"\textbf{Matriz diagonalizável em todo espaço.} \\ Sem certificado de não-diagonalização."))
        return

    # Gera forma de Jordan simbólica
    P, J = M.jordan_form()
    display(Math(r"\textbf{Certificado de não-diagonalização: forma de Jordan }J:"))
    display(Math(sp.latex(J)))
    display(Math(r"\textbf{Transformação de similaridade: }P^{-1} M P = J\textbf{, onde P é:}"))
    display(Math(sp.latex(P)))
    return P, J

TypeError: 'ImmutableDenseNDimArray' object is not callable